In [6]:
import pandas as pd
from pandas import DataFrame
import numpy as np
# 使用chardet库自动检测编码
import chardet

# 检测文件编码
with open("data.csv", 'rb') as file:
    encoding = chardet.detect(file.read())['encoding']

# 使用检测到的编码读取
primitive_data = pd.read_csv("data.csv", encoding=encoding)
primitive_data

,from_dev,from_port,from_city,from_level,to_dev,to_port,to_city,to_level,traffic,bandwidth
0,47,71,通辽,一般节点,1756,585,北京,网络核心,49636052613,1.000000e+11
1,47,74,通辽,一般节点,1756,776,北京,网络核心,50056871412,1.000000e+11
2,47,240,通辽,一般节点,1756,802,北京,网络核心,49453581081,1.000000e+11
3,47,241,通辽,一般节点,1997,464,天津,网络核心,49733361585,1.000000e+11
4,47,242,通辽,一般节点,474,672,哈尔滨,一般节点,50492573662,1.000000e+11
...,...,...,...,...,...,...,...,...,...,...
1113,1129,546,上海,网络核心,2050,502,石家庄,网络核心,48731433404,1.000000e+11
1114,1129,514,上海,网络核心,2473,946,吉林,一般节点,50060666120,1.000000e+11
1115,36036,499,长春,一般节点,1257,178,上海,网络核心,50545082113,1.000000e+11
1116,36422,346,天津,网络核心,1997,41,天津,网络核心,50628787089,1.000000e+11


In [7]:
primitive_data_1=primitive_data.dropna(how='any')
primitive_data_1

,from_dev,from_port,from_city,from_level,to_dev,to_port,to_city,to_level,traffic,bandwidth
0,47,71,通辽,一般节点,1756,585,北京,网络核心,49636052613,1.000000e+11
1,47,74,通辽,一般节点,1756,776,北京,网络核心,50056871412,1.000000e+11
2,47,240,通辽,一般节点,1756,802,北京,网络核心,49453581081,1.000000e+11
3,47,241,通辽,一般节点,1997,464,天津,网络核心,49733361585,1.000000e+11
4,47,242,通辽,一般节点,474,672,哈尔滨,一般节点,50492573662,1.000000e+11
...,...,...,...,...,...,...,...,...,...,...
1113,1129,546,上海,网络核心,2050,502,石家庄,网络核心,48731433404,1.000000e+11
1114,1129,514,上海,网络核心,2473,946,吉林,一般节点,50060666120,1.000000e+11
1115,36036,499,长春,一般节点,1257,178,上海,网络核心,50545082113,1.000000e+11
1116,36422,346,天津,网络核心,1997,41,天津,网络核心,50628787089,1.000000e+11


In [8]:
data_before_filter=primitive_data_1
data_after_filter_1=data_before_filter.loc[data_before_filter["traffic"]!=0]
data_after_filter_2=data_after_filter_1.loc[data_after_filter_1["from_level"]=='一般节点']
data_after_filter_2

,from_dev,from_port,from_city,from_level,to_dev,to_port,to_city,to_level,traffic,bandwidth
0,47,71,通辽,一般节点,1756,585,北京,网络核心,49636052613,1.000000e+11
1,47,74,通辽,一般节点,1756,776,北京,网络核心,50056871412,1.000000e+11
2,47,240,通辽,一般节点,1756,802,北京,网络核心,49453581081,1.000000e+11
3,47,241,通辽,一般节点,1997,464,天津,网络核心,49733361585,1.000000e+11
4,47,242,通辽,一般节点,474,672,哈尔滨,一般节点,50492573662,1.000000e+11
...,...,...,...,...,...,...,...,...,...,...
1097,2473,1460,吉林,一般节点,591,586,绥化,一般节点,48409925693,1.000000e+11
1103,36036,18,长春,一般节点,3443,650,青岛,网络核心,48663350759,1.000000e+11
1104,63,6,通辽,一般节点,36036,20,长春,一般节点,50355678076,1.000000e+11
1107,36036,52,长春,一般节点,1129,171,上海,网络核心,49345226162,1.000000e+11


In [12]:
# 加权采样：to_level的值为一般节点与网络核心的权重之比为 1:5
data_before_sample = data_after_filter_2
columns = data_before_sample.columns

# 创建副本并添加权重列
weight_sample = data_before_sample.copy()
weight_sample['weight'] = 0

# 设置权重
for i in weight_sample.index:
    if weight_sample.at[i, 'to_level'] == '一般节点':
        weight = 1
    else:
        weight = 5
    weight_sample.at[i, 'weight'] = weight

# 进行加权采样（采样50个样本）
weight_sample_finish = weight_sample.sample(n=50, weights='weight', random_state=42)

# 只保留原始列（去掉添加的weight列）
weight_sample_finish = weight_sample_finish[columns]

weight_sample_finish

,from_dev,from_port,from_city,from_level,to_dev,to_port,to_city,to_level,traffic,bandwidth
292,63,6,通辽,一般节点,2841,418,郑州,网络核心,51392218854,1.000000e+11
1005,36036,499,长春,一般节点,2050,502,石家庄,网络核心,49116324777,1.000000e+11
534,47,258,通辽,一般节点,1997,84,天津,网络核心,50060087433,1.000000e+11
411,591,19,绥化,一般节点,235,1506,北京,网络核心,50171685281,1.000000e+11
80,180,200,呼和浩特,一般节点,2701,300,大连,网络核心,51884294458,1.000000e+11
32,63,282,通辽,一般节点,36422,230,天津,网络核心,49455678350,1.000000e+11
706,2473,799,吉林,一般节点,1536,86,鄂尔多斯,网络核心,49550894885,1.000000e+11
412,591,23,绥化,一般节点,2701,71,大连,网络核心,50009822342,1.000000e+11
495,47,258,通辽,一般节点,235,1958,北京,网络核心,48574009525,1.000000e+11
11,47,259,通辽,一般节点,1756,245,北京,网络核心,50703793815,1.000000e+11


In [13]:
random_sample=data_before_sample
random_sample_finish=random_sample.sample(n=50)
random_sample_finish=random_sample_finish[columns]
random_sample_finish

,from_dev,from_port,from_city,from_level,to_dev,to_port,to_city,to_level,traffic,bandwidth
396,474,1269,哈尔滨,一般节点,96,152,呼和浩特,一般节点,50191686376,1.000000e+11
410,591,17,绥化,一般节点,180,20,呼和浩特,一般节点,49921741386,1.000000e+11
365,180,260,呼和浩特,一般节点,1756,788,北京,网络核心,48917626581,1.000000e+11
1115,36036,499,长春,一般节点,1257,178,上海,网络核心,50545082113,1.000000e+11
131,474,1473,哈尔滨,一般节点,2549,1461,沈阳,网络核心,53304989080,1.000000e+11
83,180,210,呼和浩特,一般节点,2194,450,唐山,网络核心,50514699101,1.000000e+11
178,787,326,玉溪,一般节点,3213,597,重庆,网络核心,48608499709,1.000000e+11
49,96,152,呼和浩特,一般节点,47,314,通辽,一般节点,51981076188,1.000000e+11
379,474,473,哈尔滨,一般节点,474,1374,哈尔滨,一般节点,50592340509,1.000000e+11
22,63,60,通辽,一般节点,36422,258,天津,网络核心,49920786706,1.000000e+11


In [14]:
ybjd=data_before_sample.loc[data_before_sample['to_level']=='一般节点']
wlhx=data_before_sample.loc[data_before_sample['to_level']=='网络核心']
after_sample=pd.concat([ybjd.sample(17),wlhx.sample(33)])
after_sample

,from_dev,from_port,from_city,from_level,to_dev,to_port,to_city,to_level,traffic,bandwidth
785,180,252,呼和浩特,一般节点,180,252,呼和浩特,一般节点,47786098667,1.000000e+11
980,4360,472,南京,一般节点,63,286,通辽,一般节点,49837582425,1.000000e+11
762,474,1374,哈尔滨,一般节点,180,18,呼和浩特,一般节点,48043608658,1.000000e+11
311,96,105,呼和浩特,一般节点,2473,804,吉林,一般节点,51224734473,1.000000e+11
148,591,558,绥化,一般节点,36036,499,长春,一般节点,49953028308,1.000000e+11
791,180,264,呼和浩特,一般节点,180,276,呼和浩特,一般节点,49965760241,1.000000e+11
79,180,192,呼和浩特,一般节点,591,586,绥化,一般节点,49504348509,1.000000e+11
354,180,192,呼和浩特,一般节点,4360,271,南京,一般节点,51828297117,1.000000e+11
429,591,1080,绥化,一般节点,180,52,呼和浩特,一般节点,49419561900,1.000000e+11
822,47,243,通辽,一般节点,474,1311,哈尔滨,一般节点,49029906488,1.000000e+11


In [15]:
# 系统抽样：按固定间隔从总体中抽取样本
systematic_sample = data_before_sample.copy()

# 计算抽样间隔
sample_size = 50  # 需要抽取的样本数
total_size = len(systematic_sample)
k = total_size // sample_size  # 抽样间隔

# 随机选择起始点
start_point = np.random.randint(0, k)

# 生成抽样索引
selected_indices = []
for i in range(sample_size):
    index = start_point + i * k
    if index < total_size:
        selected_indices.append(index)

# 进行系统抽样
systematic_sample_finish = systematic_sample.iloc[selected_indices]
systematic_sample_finish

,from_dev,from_port,from_city,from_level,to_dev,to_port,to_city,to_level,traffic,bandwidth
10,47,258,通辽,一般节点,1997,122,天津,网络核心,49594312223,1.000000e+11
21,63,58,通辽,一般节点,36036,54,长春,一般节点,48363382095,1.000000e+11
32,63,282,通辽,一般节点,36422,230,天津,网络核心,49455678350,1.000000e+11
43,96,124,呼和浩特,一般节点,47,243,通辽,一般节点,49986988230,1.000000e+11
54,96,159,呼和浩特,一般节点,2360,266,太原,网络核心,51625089370,1.000000e+11
65,180,20,呼和浩特,一般节点,63,224,通辽,一般节点,50551711152,1.000000e+11
76,180,90,呼和浩特,一般节点,235,1958,北京,网络核心,50714891315,1.000000e+11
87,180,252,呼和浩特,一般节点,63,12,通辽,一般节点,49137975001,1.000000e+11
98,474,417,哈尔滨,一般节点,1997,41,天津,网络核心,51874083489,1.000000e+11
113,474,678,哈尔滨,一般节点,1997,124,天津,网络核心,49044545927,1.000000e+11


In [16]:
# 整群抽样：按to_level进行分群，随机选择群
cluster_sample = data_before_sample.copy()

# 获取所有的群（to_level的唯一值）
clusters = cluster_sample['to_level'].unique()
print("可用的群:", clusters)

# 随机选择1个群
selected_cluster = np.random.choice(clusters, size=1, replace=False)[0]
print("被选中的群:", selected_cluster)

# 选择属于这个群的所有样本
cluster_sample_finish = cluster_sample[cluster_sample['to_level'] == selected_cluster]
cluster_sample_finish

可用的群: ['网络核心' '一般节点']
被选中的群: 一般节点


,from_dev,from_port,from_city,from_level,to_dev,to_port,to_city,to_level,traffic,bandwidth
4,47,242,通辽,一般节点,474,672,哈尔滨,一般节点,50492573662,1.000000e+11
5,47,243,通辽,一般节点,96,124,呼和浩特,一般节点,49942713747,1.000000e+11
7,47,250,通辽,一般节点,2473,762,吉林,一般节点,49108721007,1.000000e+11
9,47,252,通辽,一般节点,96,134,呼和浩特,一般节点,50256475808,1.000000e+11
13,47,314,通辽,一般节点,96,152,呼和浩特,一般节点,50161220081,1.000000e+11
...,...,...,...,...,...,...,...,...,...,...
1057,47,243,通辽,一般节点,2473,769,吉林,一般节点,49117847542,1.000000e+11
1063,47,314,通辽,一般节点,47,252,通辽,一般节点,49900452417,1.000000e+11
1079,63,224,通辽,一般节点,4069,1196,宁波,一般节点,50209459772,1.000000e+11
1097,2473,1460,吉林,一般节点,591,586,绥化,一般节点,48409925693,1.000000e+11


In [17]:
# 五种抽样方法结果比较 
# 创建抽样方法结果字典
sampling_results = {
    "加权抽样": weight_sample_finish,
    "简单随机抽样": random_sample_finish,
    "分层抽样": after_sample,
    "系统抽样": systematic_sample_finish,
    "整群抽样": cluster_sample_finish
}

# 比较各抽样方法的基本统计信息
comparison_data = []

for method_name, sample_data in sampling_results.items():
    sample_size = len(sample_data)
    
    # 计算to_level的分布
    to_level_counts = sample_data['to_level'].value_counts()
    
    comparison_data.append({
        '抽样方法': method_name,
        '样本量': sample_size,
        '流量均值': round(sample_data['traffic'].mean(), 2),
        '流量标准差': round(sample_data['traffic'].std(), 2),
        '一般节点数量': to_level_counts.get('一般节点', 0),
        '网络核心数量': to_level_counts.get('网络核心', 0),
        '一般节点比例': round(to_level_counts.get('一般节点', 0) / sample_size * 100, 1)
    })

# 创建比较表格
comparison_df = pd.DataFrame(comparison_data)
print("五种抽样方法结果比较:")
print(comparison_df.to_string(index=False))

comparison_df

五种抽样方法结果比较:
  抽样方法  样本量         流量均值        流量标准差  一般节点数量  网络核心数量  一般节点比例
  加权抽样   50 5.007086e+10 1.074005e+09       0      50     0.0
简单随机抽样   50 5.006416e+10 1.112589e+09      19      31    38.0
  分层抽样   50 4.988252e+10 1.080898e+09      17      33    34.0
  系统抽样   50 5.002733e+10 1.073238e+09      19      31    38.0
  整群抽样  186 4.996351e+10 9.839764e+08     186       0   100.0


,抽样方法,样本量,流量均值,流量标准差,一般节点数量,网络核心数量,一般节点比例
0,加权抽样,50,5.007086e+10,1.074005e+09,0,50,0.0
1,简单随机抽样,50,5.006416e+10,1.112589e+09,19,31,38.0
2,分层抽样,50,4.988252e+10,1.080898e+09,17,33,34.0
3,系统抽样,50,5.002733e+10,1.073238e+09,19,31,38.0
4,整群抽样,186,4.996351e+10,9.839764e+08,186,0,100.0
